# Free Response Answer Type

This notebook demonstrates how to generate open-ended questions with free-form text answers. Free response questions are perfect for open-ended analysis, explanation generation, and creative forecasting where you need detailed, nuanced answers.

In [ ]:
%pip install -e ..
%pip install dotenv

from IPython.display import clear_output
clear_output()

In [ ]:
import os
from dotenv import load_dotenv
from lightningrod import LightningRod

load_dotenv()

api_key = os.getenv("LIGHTNINGROD_API_KEY")
base_url = os.getenv("LIGHTNINGROD_BASE_URL", "https://api.lightningrod.ai/api/public/v1")

if not api_key:
    raise ValueError("LIGHTNINGROD_API_KEY is not set")

# Note: base_url param can be omitted
lr = LightningRod(api_key=api_key, base_url=base_url)

## Free Response Answer Type Overview

Free response questions return full text answers, not just single values or choices. They're ideal for:
- Open-ended analysis and explanations
- Creative forecasting
- Detailed scenario descriptions
- Questions that require nuanced, multi-sentence answers

Free response questions work with any data source: Google News, GDELT, or custom documents.

In [ ]:
from datetime import datetime, timedelta
from lightningrod import AnswerType, AnswerTypeEnum, NewsSeedGenerator, QuestionGenerator, WebSearchLabeler, QuestionRenderer, QuestionPipeline

answer_type = AnswerType(answer_type=AnswerTypeEnum.FREE_RESPONSE)

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    search_query="technology industry trends",
)

question_generator = QuestionGenerator(
    instructions=(
        "Generate open-ended forecasting questions that require detailed explanations. "
        "Questions should ask for analysis, predictions, or explanations that require multiple sentences to answer."
    ),
    examples=[
        "What will be the main theme of Apple's next product launch?",
        "Explain the potential impact of the new AI regulation on tech companies.",
        "What trends will dominate the technology industry in 2025?",
        "Describe how the recent merger will affect the competitive landscape.",
    ],
    bad_examples=[
        "Will Apple release a new product?",
        "What is AI?",
        "When will the product launch?",
    ],
    answer_type=answer_type,
)

# Labeler automatically finds answers to questions using web search
labeler = WebSearchLabeler(
    answer_type=answer_type,
    confidence_threshold=0.5,
)

pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=QuestionRenderer(answer_type=answer_type),
)

## Run the Pipeline

Generate free response questions from the news articles.

In [ ]:
dataset = lr.transforms.run(pipeline_config, max_questions=10) # keep max questions low when testing

> Note: This can take a few minutes to complete processing.

## View Results

Inspect the generated questions and answers. Each sample contains `seed`, `question`, `label`, `prompt`, and optional `context` and `meta` fields. See [API.md](../API.md) for the complete sample structure.

In [7]:
%pip install pandas

from IPython.display import clear_output
clear_output()

In [ ]:
import pandas as pd

# Download samples to memory
samples = dataset.download()
print(f"Generated {dataset.num_rows} samples\n")

# Convert cached samples to a list of dictionaries
rows = dataset.flattened()

df = pd.DataFrame(rows)
df.head()

Generated 10 samples

                              question.question_text  \
0  Analyze how the transition of generative AI fr...   
1  Analyze how the implementation of the One Big ...   
2  Based on the emerging AI trends for 2026 menti...   
3  How will the anticipated integration of genera...   
4  How will the increasing requirement for formal...   

                                         label.label  label.label_confidence  \
0  By 2026, the transition of generative AI from ...                     0.9   
1  The implementation of the One Big Beautiful Bi...                     1.0   
2  The intersection of emerging AI trends like So...                     0.9   
3  The integration of generative AI in retail dur...                     0.9   
4                                       Undetermined                     0.8   

                                     label.reasoning  \
0  The provided text comprehensively analyzes how...   
1  The existence and specific provisions of the 

## Use Cases for Free Response Questions

**Open-Ended Analysis:**
- "Explain the impact of X on Y"
- "What are the implications of Z?"

**Creative Forecasting:**
- "What will be the main theme of the next product launch?"
- "Describe how the industry will evolve"

**Detailed Predictions:**
- "What trends will dominate?"
- "How will the market change?"